# TP TAA


lien git : https://github.com/tyrosine-rex/tp_taa_apprentissage_supervisee

In [ ]:
from warnings import filterwarnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn import set_config
set_config(display="diagram")

import lib.taa_tools as taa

np.set_printoptions(threshold=10000,suppress=True) 
filterwarnings('ignore')

CREDIT_SCORE = "../data/credit_scoring.csv"
CREDIT_SCORE_HET = "../data/credit.data"
SPAM = "../data/SMSSpamCollection.data"
YELP = "../data/yelp-text-by-stars.csv"

# I) Apprentissage supervisé : **Feature engineering et Classification**

## 1) Chargement des données et préparation :

In [ ]:
df_credit = pd.read_csv(CREDIT_SCORE, sep=";")

In [ ]:
taa.explore_data(df_credit, "df_credit")

Le jeu de donnée 'credit_scoring.csv' présente 14 colonnes/variables et 4375 lignes/enregistrements. Chaque enregistrement est une demande de crédit, pour chaque demandeur est renseigné 13 variables d'entrée et 1 variable de sortie. Toutes les variables sont de natures numériques (certaines sont binaires). 
La colonne 'Status' est la variable de sortie que l'on va essayer de prédire à partir des variables d'entrée. Son codage correspond à:
- 1: Solvable (72.21%)
- 0: Non-solvable (27.79%)

In [ ]:
acc_prior = taa.calc_prior_accuracy(df_credit["Status"])

In [ ]:
from sklearn.model_selection import train_test_split

X, Y = df_credit.values[:, :-1], df_credit.values[:, -1]
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.5, random_state=1)

## 2) Apprentissage et évaluation de modèles :

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

COLS_EVAL = ["method", "preprocess", "precision", "accuracy", "recall", "VP", "VN", "FP", "FN", "args"]
DATAS=(xtrain, ytrain, xtest, ytest)

# tests
cart_test = taa.test_model(*DATAS, DecisionTreeClassifier, random_state=1)
knn_test = taa.test_model(*DATAS, KNeighborsClassifier, n_neighbors=5)
mlp_test = taa.test_model(*DATAS, MLPClassifier, hidden_layer_sizes=(40, 20), random_state=1)

df_comparative = pd.concat([cart_test, knn_test, mlp_test], ignore_index=True).drop_duplicates()

On définit la **precision** (ou **positive predictive value**): $\frac{VP}{VP+FP}$ "Parmis les prédictions positives, lesquelles sont justes/vraies"

On définit l'**accuracy** : $\frac{VP+VN}{VP+FP+VN+FN}$ "Parmis l'ensemble des prédictions, lesquelles sont justes"

On définit le **recall** (ou **sensibilité** ou **true positive rate**): $\frac{VP}{VP+FN}$ "Parmis les cas postifs, lesquelles sont prédites"

Dans notre situation, nous représentons l'assureur, nous voulons éviter les impayés ! il faut alors minimiser au possible les **faux positifs** (c'est à dire ne pas attribuer un prêt alors qu'un agent ne l'aurait pas fait). 
Ainsi le critère de **précision** est plus important que le recall puisque son calcul prend en compte les faux positifs (la précision augment à mesure que les FP diminue).

Dans ce premier test, c'est la méthode par arbre de décision qui a la plus basse precision (0.82 c'est le plus prudent) mais au prix d'une nombre plus faible de call positif (et par conséquent moins de client).
Bien que toutes les méthodes ont une accuracy quasi-similaire, leurs décisions sont très différentes à en regarder les matrices de confusion associée.

## 3) Normalisation des variables continues :

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Standard scaling datas
stdScale=StandardScaler()
stdScale.fit(xtrain)
xtrain_ss, xtest_ss = stdScale.transform(xtrain), stdScale.transform(xtest)

# Min max scaling datas
mmScale= MinMaxScaler()
mmScale.fit(xtrain)
xtrain_mm, xtest_mm = mmScale.transform(xtrain), mmScale.transform(xtest)

DATAS_SS = (xtrain_ss, ytrain, xtest_ss, ytest)
DATAS_MM = (xtrain_mm, ytrain, xtest_mm, ytest)

# test
cart_test_ss = taa.test_model(*DATAS_SS, DecisionTreeClassifier, random_state=1, preprocess="StdScale")
cart_test_mm = taa.test_model(*DATAS_MM, DecisionTreeClassifier, random_state=1, preprocess="MinMaxScale")
knn_test_ss = taa.test_model(*DATAS_SS, KNeighborsClassifier, n_neighbors=5, preprocess="StdScale")
knn_test_mm = taa.test_model(*DATAS_MM, KNeighborsClassifier, n_neighbors=5, preprocess="MinMaxScale")
mlp_test_ss = taa.test_model(*DATAS_SS, MLPClassifier, hidden_layer_sizes=(40, 20), random_state=1, preprocess="StdScale")
mlp_test_mm = taa.test_model(*DATAS_MM, MLPClassifier, hidden_layer_sizes=(40, 20), random_state=1, preprocess="MinMaxScale")

df_comparative = pd.concat([df_comparative, 
        cart_test_ss, knn_test_ss, mlp_test_ss,
        cart_test_mm, knn_test_mm, mlp_test_mm
        ], ignore_index=True).drop_duplicates()

Ici on procède à des normalisations de données:
- **StandardScaler()**: Centre et réduit les données pour chaque colonne
- **MinMaxScaler()**: Normalise les données dans un intervalle de 0 et 1 pour chaque colonne

De manière globale, la StandardScale est plus interressante car elle augmente plus la precision que la MinMaxScale

Nous discuterons de ces préproccessing à la fin de la partie "*I)5) Sélection de variables*"

## 4) Création de nouvelles variables caractéristiques par combinaisons linéaires des variables initiales :

In [ ]:
from sklearn.decomposition import PCA

# PCA computation
pca = PCA()
pca.fit(xtrain_ss)
pca_train, pca_test = pca.transform(xtrain_ss), pca.transform(xtest_ss)

# bind the 3 first PCA axis to std scalled x datas
xtrain_ss_pca = np.column_stack((xtrain_ss, pca_train[:, :3]))
xtest_ss_pca = np.column_stack((xtest_ss, pca_test[:, :3]))

DATAS_SS_PCA = (xtrain_ss_pca, ytrain, xtest_ss_pca, ytest)

# test
cart_test_ss_pca = taa.test_model(*DATAS_SS_PCA, DecisionTreeClassifier, random_state=1, preprocess="StdScale + PCA")
knn_test_ss_pca = taa.test_model(*DATAS_SS_PCA, KNeighborsClassifier, n_neighbors=5, preprocess="StdScale + PCA")
mlp_test_ss_pca = taa.test_model(*DATAS_SS_PCA, MLPClassifier, hidden_layer_sizes=(40, 20), random_state=1, preprocess="StdScale + PCA")

df_comparative = pd.concat([df_comparative, cart_test_ss_pca, knn_test_ss_pca, mlp_test_ss_pca], ignore_index=True).drop_duplicates()

La **PCA().fit()** nous permet:
- Identifier les facteurs d'inerties (combinaison linéaire de toutes les variables) avec nos données. Ces facteurs constituent la base d'un espace où la dispersion des données est maximale.
- Classer ces facteurs d'inerties par leurs participations à la variance des données.

La **PCA().transform()** nous permet:
- Trouve les coordonnées de nos données dans un nouvel espace (dont la base sont les facteurs d'inerties trouvés). On ne garde que celles des 3 premiers facteurs d'inertie)

Ainsi on devrait **augmenter le signal/bruit** (car on prends les facteurs les plus explicatifs) ce qui devrait aider nos prédictions.

Nous discuterons de ces préproccessing à la fin de la partie "*I)5) Sélection de variables*"

## 5) Sélection de variables :

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, random_state = 1)
clf.fit(xtrain_ss, ytrain)
importances=clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],axis=0)
sorted_idx = np.argsort(importances)[::-1]
features = df_credit.columns.values
print(features[sorted_idx])
padding = np.arange(xtrain_ss.size/len(xtrain_ss)) + 0.5 
plt.barh(padding, importances[sorted_idx],xerr=std[sorted_idx], align='center') 
plt.yticks(padding, features[sorted_idx]) 
plt.xlabel("Relative Importance")
plt.title("Variable Importance") 
plt.show()

In [ ]:
MODELS = {
    "CART": DecisionTreeClassifier(random_state=1),
    "MLP" : MLPClassifier(hidden_layer_sizes=(40, 20), random_state=1),
    "KNN" : KNeighborsClassifier(n_neighbors=5)
}

trim_test = taa.test_trimming(*DATAS_SS, MODELS, sorted_idx)

taa.results_test_trimming(trim_test)

Dans chacun des cas (ou presque) on atteint soit un pic (MLP) soit un plateau (KNN et CART) d'accuracy à partir de 7 variables incluses (rank par leurs importances).
Après ce seuil, il y a une baisse d'accuracy -> effet du bruit sur les données.

In [ ]:
# trimming
SIGNIFICANCE_COLS = sorted_idx[:8]
X_trimmed = X[:, SIGNIFICANCE_COLS]

xtrain_trim, xtest_trim, ytrain, ytest = train_test_split(X_trimmed, Y, test_size=0.5, random_state=1)

# Stdscale
stdScale=StandardScaler()
stdScale.fit(xtrain_trim)

#Trim + stdScale
xtrain_trim_ss = stdScale.transform(xtrain_trim)
xtest_trim_ss = stdScale.transform(xtest_trim)

#Trim + stdScale + PCA
xtrain_trim_ss_pca = np.column_stack((xtrain_trim_ss, pca_train[:, :3]))
xtest_trim_ss_pca = np.column_stack((xtest_trim_ss, pca_test[:, :3]))


DATAS_TRIM_SS_PCA = (xtrain_trim_ss_pca, ytrain, xtest_trim_ss_pca, ytest)
DATAS_TRIM_SS = (xtrain_trim_ss, ytrain, xtest_trim_ss, ytest)

# test Trim + stdScale
cart_test_trim_ss = taa.test_model(*DATAS_TRIM_SS, DecisionTreeClassifier, random_state=1, preprocess="Trim + StdScale")
knn_test_trim_ss = taa.test_model(*DATAS_TRIM_SS, KNeighborsClassifier, n_neighbors=5, preprocess="Trim + StdScale")
mlp_test_trim_ss = taa.test_model(*DATAS_TRIM_SS, MLPClassifier, hidden_layer_sizes=(40, 20), random_state=1, preprocess="Trim + StdScale")

# test Trim + stdScale + PCA
cart_test_trim_ss_pca = taa.test_model(*DATAS_TRIM_SS_PCA, DecisionTreeClassifier, random_state=1, preprocess="Trim + StdScale + PCA")
knn_test_trim_ss_pca = taa.test_model(*DATAS_TRIM_SS_PCA, KNeighborsClassifier, n_neighbors=5, preprocess="Trim + StdScale + PCA")
mlp_test_trim_ss_pca = taa.test_model(*DATAS_TRIM_SS_PCA, MLPClassifier, hidden_layer_sizes=(40, 20), random_state=1, preprocess="Trim + StdScale + PCA")


df_comparative = pd.concat([df_comparative, 
        cart_test_trim_ss, knn_test_trim_ss, mlp_test_trim_ss,
        cart_test_trim_ss_pca, knn_test_trim_ss_pca, mlp_test_trim_ss_pca
        ], ignore_index=True).drop_duplicates()

In [ ]:
print("Comparatif des preprocessing:")
df_comparative.sort_values("precision", ascending=False)

Ici, on peut voir que de manière globale, les MLP s'en sorte mieux (en terme de precision) que les autres, à condition d'avoir un préprocessing.

In [ ]:
taa.comparative_preprocessing(df_comparative)

Ici, on peut remarquer trouver le meilleur preprocessing pour chaque type de modèle, en fonction de nos besoin (precision):
- **TRIMMING + StandardScale** pour MLP et KNN
- Et **StandardScale + PCA** pour CART 

In [ ]:
df_comparative.pivot_table(
    values=['accuracy', 'precision', 'recall'], 
    index=["preprocess"], 
    aggfunc=np.mean).sort_values("precision", ascending=False)

Et enfin, on peut ordonner les meilleures techniques de preprocessing, celle qui marche le mieux en moyenne du point de vue de la precision:

1. Trim + StdScale	
2. StdScale + PCA	
3. Trim + StdScale + PCA	
4. StdScale	
5. MinMaxScale
6. (aucun)

## Paramétrage des classifieurs :

In [ ]:
from sklearn.model_selection import GridSearchCV


@taa.cache_pickle  #to save the result inside ./res/__cache_pickle__
def find_best_param(sk_function, params, X_train, Y_train):
    gs = GridSearchCV(sk_function(), params, scoring="precision") #we target the best precision score)
    gs.fit(X_train, Y_train)
    return gs


parameters_knn = {
    "n_neighbors" : [i for i in range(1, 100, 5)],
    "weights" : ["uniform", "distance"],
    "algorithm" : ['auto', 'ball_tree', 'kd_tree', 'brute']
}

parameters_cart = {
    "criterion" : ["gini", "entropy", "log_loss"],
    "splitter" : ["best", "random"],
    "max_depth" : [i for i in range(1, 25)]
}

parameters_mlp = {
    "activation" : ["identity", "logistic", "tanh", "relu"],
    "hidden_layer_sizes" : [(L1, L2) for L1 in range(15, 121, 15) for L2 in range(int(L1*0.2), int(L1*0.8)+1, int(L1/5))]
}

#grid search
gs_KNN = find_best_param(KNeighborsClassifier, parameters_knn, xtrain_trim_ss, ytrain)
gs_CART = find_best_param(DecisionTreeClassifier, parameters_cart, xtrain_ss_pca, ytrain)
gs_MLP = find_best_param(MLPClassifier, parameters_mlp, xtrain_trim_ss, ytrain)

print("KNN", gs_KNN.best_params_, "with precision:", gs_KNN.best_score_)
print("CART", gs_CART.best_params_, "with precision:", gs_CART.best_score_)
print("MLP", gs_MLP.best_params_, "with precision:", gs_MLP.best_score_)

|modèle|preprocessing|params|
|---|---|---|
|KNN|trim+stdScale|{'algorithm': 'auto', 'n_neighbors': 6, 'weights': 'uniform'}|
|MLP|trim+stdScale|{'activation': 'relu', 'hidden_layer_sizes': (105, 42)}|
|CART|stdScale+PCA|{'criterion': 'entropy', 'max_depth': 4, 'splitter': 'best'}|

## Création d'un pipeline :

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from pickle import load, dump


@taa.cache_pickle  #to save the result into ./res/__cache_pickle__
def fit_pipeline(pipeline, X_train, Y_train):
    pipeline.fit(X_train, Y_train)
    return pipeline

# CART preprocess
STD_PCA = Pipeline([
    ("stdScale", StandardScaler()),
    ("pca", PCA(n_components=3))
])

STDwithSTD_PCA=FeatureUnion([
    ("stdScale", StandardScaler()),
    ("std+pca", STD_PCA)
])

# CART
CART_PIPE=Pipeline([
    ("preprocess", STDwithSTD_PCA),
    ("estimator", DecisionTreeClassifier(**gs_CART.best_params_))
])

# KNN and MLP preprocess
TRIM = ColumnTransformer([
    ("trim", "passthrough", SIGNIFICANCE_COLS)
])

TRIM_STD=Pipeline([
    ("trim", TRIM),
    ("stdScale", StandardScaler())
])

# MLP
MLP_PIPE=Pipeline([
    ("preprocess", TRIM_STD),
    ("estimator", MLPClassifier(**gs_MLP.best_params_))
])

# KNN
KNN_PIPE=Pipeline([
    ("preprocess", TRIM_STD),
    ("estimator", KNeighborsClassifier(**gs_KNN.best_params_))
])

# train pipelines with RAW DATA
CART_PIPE = fit_pipeline(CART_PIPE, xtrain, ytrain)
MLP_PIPE = fit_pipeline(MLP_PIPE, xtrain, ytrain)
KNN_PIPE = fit_pipeline(KNN_PIPE, xtrain, ytrain)

# save it with pickle
dump(CART_PIPE, open("../res/pipe_cart.pkl", "wb"))
dump(MLP_PIPE, open("../res/pipe_mlp.pkl", "wb"))
dump(KNN_PIPE, open("../res/pipe_knn.pkl", "wb"))

del KNN_PIPE, CART_PIPE, MLP_PIPE

# load it with pickle
CART_PIPE = load(open("../res/pipe_cart.pkl", "rb"))
MLP_PIPE = load(open("../res/pipe_mlp.pkl", "rb"))
KNN_PIPE = load(open("../res/pipe_knn.pkl", "rb"))

display(CART_PIPE)
display(MLP_PIPE)
display(KNN_PIPE)

In [ ]:
# test pipelines
test_pipe = taa.test_pipeline([CART_PIPE, MLP_PIPE, KNN_PIPE], xtest, ytest)
display(test_pipe)

Paradoxallement, en utilisant les paramètres trouvés par **GridSearchCV()**, en faisant les mêmes préprocessing on obtient des resultats en precision moins bon qu'avec celui de la partie *I)5)*.

## Comparaisons de plusieurs algorithmes d'apprentissages :

In [ ]:
from IPython.display import Markdown

txt = "|**Key**|**Classifiers**|\n|--|--|\n"
for k, v in taa.CLASSIFIERS.items():
    txt += f"|{k}|{v}|\n"

Markdown(txt)

In [ ]:
run1=taa.run_classifiers(taa.CLASSIFIERS, xtrain_trim_ss, ytrain)
taa.results_run_clfs(run1, "run1", top=12)

D'après le 'run1' le mieux serait de partir sur:
- un modèle type **MultiLayersPerceptron** 
- un modèle type **AdaBoost**

Il présentent les meilleures compromis en terme de precision et d'accuracy.

# Apprentissage supervisé : Données hétérogènes

## Chargement des données et préparation

In [ ]:
df_credit_het = pd.read_csv(CREDIT_SCORE_HET, sep="\t", header=None, na_values="?")

In [ ]:
taa.explore_data(df_credit_het, "def_credit_het")

Le jeu de donnée 'credit.data' présente 16 colonnes/variables et 688 lignes/enregistrements.

Chaque enregistrement est une demande de crédit, pour chaque demandeur est renseigné 13 variables d'entrée et 1 variable de sortie. On distingues deux types de variables:
- **Numériques** : continues ou discrètes -> 1, 2, 7, 10, 13, 14 + **15**
- **Catégorielles** : -> 0, 3, 4, 5, 6, 8, 9, 11, 12

La colonne '**15**' est la variable de sortie que l'on va essayer de prédire à parir des variables d'entrée. 
Elle a deux modalités:
- "+" / 1 : (44.33%)
- "-" / 0 : (55.67%)

In [ ]:
acc_prior = taa.calc_prior_accuracy(df_credit_het[15])

In [ ]:
CAT_COLS = [0, 3, 4, 5, 6, 8, 9, 11, 12]
NUM_COLS = [1, 2, 7, 10, 13, 14]

# to numpy array
X, Y = df_credit_het.values[:, :15], df_credit_het.values[:, 15]

# separate into X_num, X_cat and Y arrays
X_num = X[:, NUM_COLS]
X_num = X_num.astype(np.float64)

X_cat = X[:, CAT_COLS]

Y[Y == "+"] = 1
Y[Y == "-"] = 0
Y = Y.astype(np.float64)

# without NA
without_na = ~np.isnan(X_num).any(axis=1)
X_num_nona = X_num[without_na]
Y_nona = Y[without_na]

## 2) Normalisation des variables continues :

Par continues on entend toute valeurs numériques trouvées dans le jeu de donnée.

In [ ]:
# Standard scaling X_num
stdScale=StandardScaler()
X_num_nona_ss = stdScale.fit_transform(X_num_nona)

# Min max scaling X_num
mmScale= MinMaxScaler()
X_num_nona_mm = mmScale.fit_transform(X_num_nona)

# run
run2_vanilla = taa.run_classifiers(taa.CLASSIFIERS, X_num_nona, Y_nona)
run2_ss = taa.run_classifiers(taa.CLASSIFIERS, X_num_nona_ss, Y_nona)
run2_mm = taa.run_classifiers(taa.CLASSIFIERS, X_num_nona_mm, Y_nona)

taa.results_run_clfs(run2_vanilla, "run2_vanilla", scores=["test_accuracy", "test_roc_auc"])
taa.results_run_clfs(run2_mm, "run2_mm", scores=["test_accuracy", "test_roc_auc"])
taa.results_run_clfs(run2_ss, "run2_ss", scores=["test_accuracy", "test_roc_auc"])

La **ROC AUC** est l'aire sous la courbe (AUC) d'un ROC, une courbe qui rend compte de la sensibilité/recall(en y) et de la 1-spécificité(en x). Plus cette valeur est proche de 1, meilleur est le classifieur.

On remarque que les valeurs de ROC fluctuent peu entre run2_vanilla, run2_mm et run2_ss. Idem pour les accuracy. Ce qui change c'est le type de classifieurs qui va se retrouver en tête. On retrouve des RandomForrest, des MultiLayersPerceptron, des AdaBoosts et des Baggings (mais les Baggings ils sont lents à fit)

## Traitement de données manquantes : 

In [ ]:
from sklearn.impute import SimpleImputer

# for categorical data
X_cat_imp = np.copy(X_cat)
X_cat_imp[pd.isna(X_cat_imp)] = "NA" #cause np.nan is a float 
for i in range(len(CAT_COLS)):       
    unique_val, val_idx = np.unique(X_cat_imp[:, i], return_inverse=True) 
    X_cat_imp[:, i] = val_idx

imp_cat = SimpleImputer(missing_values="NA", strategy='most_frequent')                                                                           
X_cat_imp[:, range(5)] = imp_cat.fit_transform(X_cat_imp[:, range(5)])

#  for numerical data
X_num_imp = np.copy(X_num)
imp_num = SimpleImputer(missing_values=np.nan, strategy='mean') 
X_num_imp = imp_num.fit_transform(X_num_imp)

Pour les variables 'catégorielles', on remplace chaque modalités par un nombre entier, et ce dans chaque colonne. Les valeurs "NA" sont remplacé par la modalité la plus présente dans la colonne.

Pour les variables 'numériques', on remplace chaque "NA" par la moyenne de la colonne.

Mais est-ce que cela ne rajoute t'il pas du bruit ?

## Traitement de variables catégorielles :

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
X_cat_imp_bin = encoder.fit_transform(X_cat_imp).toarray()

Ici, **OneHotEncoder()** va transformer les modalités multiclasse en modalité binaire, elle ajoute des colonnes si nécessaire:
- si 1 colonne à 2 modalités -> crée 1 colonne binaire
- si 1 colonne à 3 modalités -> crée 2 colonne binaire
- si 1 colonne à 4 modalités -> crée 2 colonne binaire
- si 1 colonne à 5 modalités -> crée 3 colonne binaire
- ...

In [ ]:
# Standard scaling X_num_imp
stdScale=StandardScaler()
X_num_imp_ss = stdScale.fit_transform(X_num_imp)

# Min max scaling X_num_imp
mmScale= MinMaxScaler()
X_num_imp_mm = mmScale.fit_transform(X_num_imp)

# bind X_cat_imp_binary with X_num_imp
X_combined = np.concatenate((X_num_imp, X_cat_imp_bin), axis=1)
X_combined_mm = np.concatenate((X_num_imp_ss, X_cat_imp_bin), axis=1)
X_combined_ss = np.concatenate((X_num_imp_mm, X_cat_imp_bin), axis=1)

# run
run3_vanilla = taa.run_classifiers(taa.CLASSIFIERS, X_combined, Y)
run3_ss = taa.run_classifiers(taa.CLASSIFIERS, X_combined_mm, Y)
run3_mm = taa.run_classifiers(taa.CLASSIFIERS, X_combined_ss, Y)

taa.results_run_clfs(run3_vanilla, "run3_imp_vanilla", scores=["test_accuracy", "test_roc_auc"])
taa.results_run_clfs(run3_ss, "run2_imp_ss", scores=["test_accuracy", "test_roc_auc"])
taa.results_run_clfs(run3_mm, "run2_imp_mm", scores=["test_accuracy", "test_roc_auc"])

Grand bond en avant en ce qui concerne la ROC AUC et l'accuracy, avec l'ajout des variables catégorielles.
Se dispute le podium les RandomForrest et les Baggings.

# Apprentissage supervisé sur des données textuelles : Feature engineering et Classification 

## Lecture et préparation des données:

In [ ]:
df_spam = pd.read_csv(SPAM, sep = "\t", header = None)
df_spam.rename(columns = {0 : "Status", 1 : "Value"}, inplace=True)

In [ ]:
taa.explore_data(df_spam, "df_spam", DPI=400)

Le jeu de donnée 'SMSSpamCollection.data' est constitué de 5572 lignes et 2 colonnes/variables.

- var 'Values' : unique pour la grande majorité des lignes (5169 valeurs uniques)
- var 'Status' : à prédire, deséquilibré avec ~85% sont en "ham" et ~15% en "spam"

Dans le cadre des spams que veut on ? Souhaitons nous tous les bloquer (recall max) ou bien on veut être sûr de ne pas se tromper (precision max)...

In [ ]:
acc_prior = taa.calc_prior_accuracy(df_spam["Status"])

In [ ]:
Y = df_spam["Status"].values
X = df_spam["Value"].values
Y[Y == "spam"] = 1
Y[Y == "ham"] = 0
Y = Y.astype(np.int64)

## CountVectorizer :

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english', min_df = 10)
X_vec = vectorizer.fit_transform(X)
X_vec = X_vec.toarray()

run4 = taa.run_classifiers(taa.CLASSIFIERS, X_vec, Y)
taa.results_run_clfs(run4, "run4", scores=["test_accuracy", "test_roc_auc"])

## Tf–idf term weighting : 

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfid = TfidfTransformer(smooth_idf=False)
X_vec_tfid = tfid.fit_transform(X_vec)
X_vec_tfid = X_vec_tfid.toarray()

run4_tfid = taa.run_classifiers(taa.CLASSIFIERS, X_vec_tfid, Y)
taa.results_run_clfs(run4_tfid, "run4_tfid", scores=["test_accuracy", "test_roc_auc"])

## TruncatedSVD :

In [ ]:
from sklearn.decomposition import TruncatedSVD

# 100 concepts
svd100 = TruncatedSVD(n_components=100, n_iter=20, random_state=1)
X_vec_tfid_svd100 = svd100.fit_transform(X_vec_tfid)

run4_tfid_svd100 = taa.run_classifiers(taa.CLASSIFIERS, X_vec_tfid_svd100, Y)
taa.results_run_clfs(run4_tfid_svd100, "run4_tfid_svd100", scores=["test_accuracy", "test_roc_auc"])

NB : 100 components ==> valeur recommandé par sklearn pour faire du 'Latent Semantic Analysis' (LSA)

"overfitting ?"

## Pipeline :

In [ ]:
from sklearn.ensemble import AdaBoostClassifier


PIPE_TEXT_ADA = Pipeline([
    ("vectorizer", CountVectorizer(stop_words='english', min_df = 10)), 
    ("tfid", TfidfTransformer(smooth_idf=False)), 
    ("svd", TruncatedSVD(n_components=100, n_iter=20, random_state=1)),
    ('estimator', AdaBoostClassifier(n_estimators=120, random_state=1))
])

PIPE_TEXT_MLP = Pipeline([
    ("vectorizer", CountVectorizer(stop_words='english', min_df = 10)),
    ("tfid", TfidfTransformer(smooth_idf=False)), 
    ("truncatedSVD", TruncatedSVD(n_components=100, n_iter=20, random_state=1)),
    ('estimator', MLPClassifier(hidden_layer_sizes=(100, 50), random_state=1))
])

display(PIPE_TEXT_ADA)
display(PIPE_TEXT_MLP)

## Application sur un autre jeu de données :

In [ ]:
df_yelp = pd.read_csv(YELP, header = 0, sep = ';', encoding='iso-8859-1')

In [ ]:
taa.explore_data(df_yelp, "df_yelp")

In [ ]:
acc_prior = taa.calc_prior_accuracy(df_yelp["Stars"])

In [ ]:
X = df_yelp["Text"].values
Y = df_yelp["Stars"].values

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.5, random_state=1)

PIPE_TEXT_ADA = fit_pipeline(PIPE_TEXT_ADA, xtrain, ytrain)
PIPE_TEXT_MLP = fit_pipeline(PIPE_TEXT_MLP, xtrain, ytrain)

#they are heavy
ypred_ADA = PIPE_TEXT_ADA.predict(xtest)
ypred_MLP = PIPE_TEXT_MLP.predict(xtest)

In [ ]:
taa.test_pipeline([PIPE_TEXT_ADA, PIPE_TEXT_MLP], xtest, ytest, score_params={'average': 'weighted'}) 

Au final ça marche pusique que l'on dépasse largement l'accuracy a du random (~0.27). Mais ce n'est pas très haut pour autant. Seulement la moitié des calls sont bons.

Ici, notre variable à prédire n'est pas binaire, elle a 5 modalités qui sont **ordinales**. Peut-être qu'un classifieur n'est pas adapté pour ce genre de modalité. Par exemple, si la valeur à prédire de 4 mais que le call soit de 1 ou de 5, l'erreur sera equivalente pour un classifieur.

Peut-être une qu'un regresseur serait plus adapté... l'ennui c'est que les modalités sont discrètes.